### Building A chatbot

In this we will cover an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remeber previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation.There are several other related concepts that you may be looking for:

* Convrsational RAG: Enable a chatbot experience over an external source of data.

* Agents: Build a chatbot that can take actions


In [39]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key= os.getenv('GROQ_API_KEY')


In [40]:
from langchain_groq import ChatGroq
model= ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000237CFA56990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000237CFA57750>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [41]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi My name is Neha and I am a chief AI engineer")])



AIMessage(content="Hello Neha, it's a pleasure to meet you!\n\nThat's an impressive title. As a chief AI engineer, I imagine you're doing some fascinating work. \n\nWhat are you currently working on that you're most excited about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 22, 'total_tokens': 78, 'completion_time': 0.101818182, 'prompt_time': 0.001337519, 'queue_time': 0.256282352, 'total_time': 0.103155701}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--53b08180-2b7b-4b6a-a182-3ea822f928e5-0', usage_metadata={'input_tokens': 22, 'output_tokens': 56, 'total_tokens': 78})

In [42]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi My name is Neha and I am a chief AI engineer"),
        AIMessage(content="Hello Neha, it's nice to meet you!\n\nThat's a fantastic role. As a large language model, I'm always interested in learning more about the work that AI engineers do.\n\nWhat kind of projects are you currently working on?  \n\nI hope you have a productive day!\n"),
        HumanMessage(content="What's my name and what do i do?")
    ]
)
# first of all we are fieding the information and asking it again toi check whether it is remebering my name. hence it is saving the message history

AIMessage(content="You said your name is Neha and that you are a chief AI engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 107, 'total_tokens': 156, 'completion_time': 0.089090909, 'prompt_time': 0.002760098, 'queue_time': 0.255034627, 'total_time': 0.091851007}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3478c490-db2e-4fb8-89dd-18c2da03c431-0', usage_metadata={'input_tokens': 107, 'output_tokens': 49, 'total_tokens': 156})

### Message History

We can use a Message history class to wrap our model and make it stateful. this will keep track of inputs and outputs of the model and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [43]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# for understanding the history of the particular user we define this function so that the llm can understand when used by multiple user at a time

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [44]:
config={"configurable":{"session_id":"chat1"}}

In [45]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Neha and I am a chief AI engineer")],
    config=config
)
response.content

"Hello Neha! It's nice to meet you.\n\nThat's a fantastic title! As a large language model, I'm always interested in learning more about the work of AI engineers.  \n\nWhat are you currently working on that you're excited about?\n"

In [46]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)
response.content

'Your name is Neha, as you told me at the beginning of our conversation. 😊  \n\nIs there anything else I can help you with?\n'

#### changing the config (session_id) to check whether it remenbers the name or not

In [47]:
config1={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config1
)
response.content

'You told me your name is Neha!  \n\nIs there something else I can help you with?\n'

In [48]:
# in abpve we are getting the response because the session_id is same
# let's change it and then check
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [49]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Sharma")],
    config=config1
)
response.content

"Hi Sharma, it's nice to meet you! 😊 \n\nIs there anything I can help you with today?\n"

In [50]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config1
)
response.content

'Your name is Sharma.  \n\nI remember! 😄  Is there anything else I can help you with?\n'

### Prompt templates

These help to turn raw user information into a fromat that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. first let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [51]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate(
    [
        ("system","You are a helpful AI assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")

    ]
)

chain=prompt|model

In [52]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is neha")]})

AIMessage(content="Hi Neha! It's nice to meet you. \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 32, 'total_tokens': 60, 'completion_time': 0.050909091, 'prompt_time': 0.001505029, 'queue_time': 0.254953621, 'total_time': 0.05241412}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f201eaa4-c5b0-42e3-bb32-1daddf548f3a-0', usage_metadata={'input_tokens': 32, 'output_tokens': 28, 'total_tokens': 60})

In [53]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [54]:
config= {"configurable": {"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Neha")],
    config=config
)
response.content

"Hello Neha, it's nice to meet you! \n\nHow can I help you today? 😊  \n\n"

In [64]:
#Add more complexity

prompt=ChatPromptTemplate(
    [
        ("system","You are a helpful AI assistant. Answer all the questions to the best of your ability in {bhasha}"),
        MessagesPlaceholder(variable_name="msg")

    ]
)
chain=prompt|model


In [56]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Neha")], "bhasha":"Hindi"})
response.content

'नमस्ते नीहा!  \n\nमैं आपकी मदद करने के लिए यहाँ हूँ।  आपके कोई सवाल हैं? 😊  \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [65]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="msg"
)

In [ ]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"msg":[HumanMessage(content="Hi i am neha")],"bhasha":"hindi"},
    config=config
)
response.content

# so for input messages key we can set it to any thing but it should be similar in all prompt with msg and invoke

'नमस्ते नेहा! 😊 \n\nआप कैसे हैं?  \n'

### Manage the Conversation History